In [1]:
from seeq import spy
import pandas as pd

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 193

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.swap

Performs "asset swapping" functionality equivalent to what you can achieve in Seeq Workbench by the "Swap in this asset..." button for an Asset found in the Data tab.

When you create a calculated item (Signal, Condition, Scalar, Threshold Metric) that is based on stored signals that are part of an asset tree, Seeq allows you to "swap" the original asset that you used for a different asset that also has all of the constituent signals (named identically).

You can then pull the data for the "swapped" calculation just like you would pull data for any other calculated item.

Let's walk through an example.

## Example

First we will query for some signals to use in our calculation and then push a calculation based on those signals.

In [3]:
area_a_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': 'Area A',
    'Name': 'Temperature'
})

pushed_items_df = spy.push(metadata=pd.DataFrame([{
            'Name': f'Added',
            'Formula': '$t + 10',
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }, {
            'Name': f'Hot',
            'Formula': '$t > 80',
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }, {
            'Name': f'Average on One Day in 2016',
            'Formula': "$t.average(capsule('2016-12-18'))",
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }]), workbook='SPy Documentation Examples >> spy.swap')

pushed_items_df

,Name,Formula,Formula Parameters,Type,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,Added,$t + 10,[t=0EF58FC2-7F42-77D0-86AC-96C7027C1771],CalculatedSignal,0EF5BF17-09A9-66C0-869E-364B7AAAD204,Seeq Data Lab,Seeq Data Lab,[0EF5BF17-09A9-66C0-869E-364B7AAAD204] {Signal...,0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5,Success
1,Hot,$t > 80,[t=0EF58FC2-7F42-77D0-86AC-96C7027C1771],CalculatedCondition,0EF5BF17-09A9-66C0-869E-364B7AAAD204,Seeq Data Lab,Seeq Data Lab,[0EF5BF17-09A9-66C0-869E-364B7AAAD204] {Condit...,0EF5BF17-0FB0-6640-B2EE-D23F5182D47A,Success
2,Average on One Day in 2016,$t.average(capsule('2016-12-18')),[t=0EF58FC2-7F42-77D0-86AC-96C7027C1771],CalculatedScalar,0EF5BF17-09A9-66C0-869E-364B7AAAD204,Seeq Data Lab,Seeq Data Lab,[0EF5BF17-09A9-66C0-869E-364B7AAAD204] {Scalar...,0EF5BF17-0F82-6010-A07C-700678029168,Success


Now we'll query for the assets to swap in for the original Area A asset that we built the calculation for:

In [4]:
areas_def_df = spy.search({
    'Path': 'Example >> Cooling Tower 2',
    'Name': '/Area [DEF]/',
    'Type': 'Asset'
}, old_asset_format=False)

areas_def_df

,Path,Name,Type,Time,Count,Pages,Result
0,Example >> Cooling Tower 2,/Area [DEF]/,Asset,00:00:00.05,3,1,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,0EF58FC2-90AA-60E0-BEE7-242FC3F918D2,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False
1,0EF58FC2-90AC-77F0-8FD2-9F21B57B5B16,Example >> Cooling Tower 2,Area E,Area E,NaN,Asset,NaN,Example Data,False
2,0EF58FC2-90AC-77F0-9C25-FE38E1BE65AC,Example >> Cooling Tower 2,Area F,Area F,NaN,Asset,NaN,Example Data,False


Now all we have to do is execute the `spy.swap()` command to swap in these assets and get a bunch of new IDs we can use in `spy.pull()`:

In [5]:
swapped_df = spy.swap(pushed_items_df, areas_def_df, errors='catalog')

,Original ID,ID,Type,Name,Result,Swap Performed,Header
0,0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5,0EF5BF17-16D0-71F0-BA8E-619B40296EEF,CalculatedSignal,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D,Cooling Tower 2 >> Area D >> Added
1,0EF5BF17-0FB0-6640-B2EE-D23F5182D47A,0EF5BF17-16D9-EE30-8188-87B17F1FF75A,CalculatedCondition,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D,Cooling Tower 2 >> Area D >> Hot
2,0EF5BF17-0F82-6010-A07C-700678029168,0EF5BF17-16D7-7720-9BDB-043D016E338C,CalculatedScalar,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D,Cooling Tower 2 >> Area D >> Average on One Day in 2016
3,0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5,0EF5BF17-170D-6280-9BF4-1FB99CD99A66,CalculatedSignal,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E,Cooling Tower 2 >> Area E >> Added
4,0EF5BF17-0FB0-6640-B2EE-D23F5182D47A,0EF5BF17-170F-F990-9DEC-60E95E721628,CalculatedCondition,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E,Cooling Tower 2 >> Area E >> Hot
5,0EF5BF17-0F82-6010-A07C-700678029168,0EF5BF17-1731-EC70-9DFE-C220A083CD0B,CalculatedScalar,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E,Cooling Tower 2 >> Area E >> Average on One Day in 2016
6,0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5,nan,CalculatedSignal,Added,"[SPyRuntimeError] Error finding swap for item ""Added"" (0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5):(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",nan,nan
7,0EF5BF17-0FB0-6640-B2EE-D23F5182D47A,nan,CalculatedCondition,Hot,"[SPyRuntimeError] Error finding swap for item ""Hot"" (0EF5BF17-0FB0-6640-B2EE-D23F5182D47A):(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",nan,nan
8,0EF5BF17-0F82-6010-A07C-700678029168,nan,CalculatedScalar,Average on One Day in 2016,"[SPyRuntimeError] Error finding swap for item ""Average on One Day in 2016"" (0EF5BF17-0F82-6010-A07C-700678029168):(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",nan,nan


You'll notice a few things about the returned DataFrame:

- The `Original ID` column corresponds to the calculated item for which we're swapping assets. The values are repeated three times because we're swapping three assets across three items (a signal, condition and scalar). That's why there's nine rows total.
- Some of the rows don't have an `ID` field because there were errors, which can be seen in the `Result` column.
- The `Swap Performed` column includes precise details about what was swapped. The format is `Swapped Out Asset --> Swapped In Asset`.

In order to get a better look at the `Result` and `Swap Performed` columns, let's remove some of the other columns:

In [6]:
pd.set_option('display.max_colwidth', None)
swapped_df[['ID', 'Name', 'Result', 'Swap Performed']]

,ID,Name,Result,Swap Performed
0,0EF5BF17-16D0-71F0-BA8E-619B40296EEF,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
1,0EF5BF17-16D9-EE30-8188-87B17F1FF75A,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
2,0EF5BF17-16D7-7720-9BDB-043D016E338C,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
3,0EF5BF17-170D-6280-9BF4-1FB99CD99A66,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E
4,0EF5BF17-170F-F990-9DEC-60E95E721628,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E
5,0EF5BF17-1731-EC70-9DFE-C220A083CD0B,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E
6,NaN,Added,"[SPyRuntimeError] Error finding swap for item ""Added"" (0EF5BF17-0F4E-FBC0-BAEB-A22BA14E8CF5):\n(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",NaN
7,NaN,Hot,"[SPyRuntimeError] Error finding swap for item ""Hot"" (0EF5BF17-0FB0-6640-B2EE-D23F5182D47A):\n(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",NaN
8,NaN,Average on One Day in 2016,"[SPyRuntimeError] Error finding swap for item ""Average on One Day in 2016"" (0EF5BF17-0F82-6010-A07C-700678029168):\n(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",NaN


In Seeq's Example data, Area F does not include a `Temperature` signal, so `spy.swap()` could not create a swapped calculation for that particular asset.

For the swaps that succeeded, you can now pull the data for these swapped items like so:

In [7]:
# First we'll drop the rows that had errors.
successful_swaps_df = swapped_df[swapped_df['Result'] == 'Success']

spy.pull(successful_swaps_df, start='2021-01-01', end='2021-01-02')

,ID,Type,Name,Header,Time,Count,Pages,Data Processed,Result
0,0EF5BF17-16D0-71F0-BA8E-619B40296EEF,CalculatedSignal,Added,Cooling Tower 2 >> Area D >> Added,00:00:00.20,97,1,11 KB,Success
1,0EF5BF17-16D9-EE30-8188-87B17F1FF75A,CalculatedCondition,Hot,Cooling Tower 2 >> Area D >> Hot,00:00:00.08,2,1,11 KB,Success
2,0EF5BF17-16D7-7720-9BDB-043D016E338C,CalculatedScalar,Average on One Day in 2016,Cooling Tower 2 >> Area D >> Average on One Day in 2016,00:00:00.26,1,0,11 KB,Success
3,0EF5BF17-170D-6280-9BF4-1FB99CD99A66,CalculatedSignal,Added,Cooling Tower 2 >> Area E >> Added,00:00:00.20,97,1,11 KB,Success
4,0EF5BF17-170F-F990-9DEC-60E95E721628,CalculatedCondition,Hot,Cooling Tower 2 >> Area E >> Hot,00:00:00.08,2,1,11 KB,Success
5,0EF5BF17-1731-EC70-9DFE-C220A083CD0B,CalculatedScalar,Average on One Day in 2016,Cooling Tower 2 >> Area E >> Average on One Day in 2016,00:00:00.26,1,0,11 KB,Success


,Cooling Tower 2 >> Area D >> Added,Cooling Tower 2 >> Area D >> Hot,Cooling Tower 2 >> Area D >> Average on One Day in 2016,Cooling Tower 2 >> Area E >> Added,Cooling Tower 2 >> Area E >> Hot,Cooling Tower 2 >> Area E >> Average on One Day in 2016
2021-01-01 00:00:00+00:00,96.583716,1.0,75.860678,101.209415,1.0,81.768867
2021-01-01 00:15:00+00:00,96.990286,1.0,75.860678,100.564875,1.0,81.768867
2021-01-01 00:30:00+00:00,97.396855,1.0,75.860678,101.359711,1.0,81.768867
2021-01-01 00:45:00+00:00,97.803424,1.0,75.860678,101.776450,1.0,81.768867
2021-01-01 01:00:00+00:00,98.209993,1.0,75.860678,101.785881,1.0,81.768867
...,...,...,...,...,...,...
2021-01-01 23:00:00+00:00,95.488781,1.0,75.860678,101.164443,1.0,81.768867
2021-01-01 23:15:00+00:00,95.957709,1.0,75.860678,102.506365,1.0,81.768867
2021-01-01 23:30:00+00:00,96.318568,1.0,75.860678,103.861894,1.0,81.768867
2021-01-01 23:45:00+00:00,97.208800,1.0,75.860678,104.494810,1.0,81.768867


## Multi-asset Swaps

Calculations can be based on more than one asset. That means swapping is more complex, because you must specify a pairing for each swappable asset. You accomplish this using "Swap Groups".

In the following example, we'll push a calculation that is a summation of `Compressor Power` for Area A and Area B. Then we'll swap out those assets for Areas D & E and then again for Areas D & F.

In [8]:
areas_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': '/Area [AB]/',
    'Name': 'Compressor Power'
})

added_together_df = spy.push(metadata=pd.DataFrame([{
    'Name': 'Added Together',
    'Formula': '$a + $b',
    'Formula Parameters': {
        '$a': areas_df[areas_df['Asset'] == 'Area A'],
        '$b': areas_df[areas_df['Asset'] == 'Area B']
    }
}]), workbook='SPy Documentation Examples >> spy.swap')

added_together_df = spy.search({
    'Name': 'Added Together'
}, include_swappable_assets=True, workbook='SPy Documentation Examples >> spy.swap')

added_together_df

,Name,Time,Count,Pages,Result
0,Added Together,00:00:00.01,1,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Swappable Assets
0,0EF5BF17-25A2-7110-8D07-FF485DDCEDB7,Added Together,NaN,CalculatedSignal,kW,Seeq Data Lab,False,ID Type Path \ 0 0EF58FC2-90AC-77F0-85F2-91128B10372A Asset Example >> Cooling Tower 1 1 0EF58FC2-90AA-60E0-94F7-29D4E89DC2F9 Asset Example >> Cooling Tower 1 Asset 0 Area A 1 Area B


Notice that we used the `spy.search(include_swappable_assets=True)` flag, which means that the `added_together_df` DataFrame includes a `Swappable Assets` column where each cell is an embedded DataFrame detailing the assets upon which the calculation is based:

In [9]:
swappable_assets_df = added_together_df.iloc[0]['Swappable Assets']
swappable_assets_df

,ID,Type,Path,Asset
0,0EF58FC2-90AC-77F0-85F2-91128B10372A,Asset,Example >> Cooling Tower 1,Area A
1,0EF58FC2-90AA-60E0-94F7-29D4E89DC2F9,Asset,Example >> Cooling Tower 1,Area B


That's what we need in order to specify our "swap groups". Now we have to create a DataFrame of assets with a `Swap Group` and `Swap Out` columns to achieve the pairing we talk about above:

In [10]:
# Grab a DataFrame that contains Areas D, E and F
cooling_tower_2_df = spy.search({
    'Path': 'Example >> Cooling Tower 2',
    'Type': 'Asset'
}, old_asset_format=False).sort_values(by='Name')

# Create a DataFrame that includes the combination of areas:
swap_groups_df = pd.DataFrame([
    cooling_tower_2_df.iloc[0],  # Area D
    cooling_tower_2_df.iloc[1],  # Area E

    cooling_tower_2_df.iloc[0],  # Area D
    cooling_tower_2_df.iloc[2]]) # Area F
    
# Now we'll specify our "Swap Group" and "Swap Out" columns. The Swap Group column can be any signifier you wish to use
# to group the swaps together. Any rows with the same value for the Swap Group will be grouped together for swapping
# purposes.
swap_groups_df['Swap Group'] = [1, 1, 2, 2]

# The "Swap Out" column can be an asset ID string, the latter part of an asset's path, or a DataFrame row for an asset.
# It must be for an asset that is part of the "Swappable Assets" found above.
swap_groups_df['Swap Out'] = [
    swappable_assets_df.iloc[0],  # Area A
    swappable_assets_df.iloc[1],  # Area B

    swappable_assets_df.iloc[0],  # Area A
    swappable_assets_df.iloc[1],  # Area B
]

swap_groups_df

,Path,Type,Time,Count,Pages,Result
0,Example >> Cooling Tower 2,Asset,00:00:00.05,3,1,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Swap Group,Swap Out
0,0EF58FC2-90AA-60E0-BEE7-242FC3F918D2,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False,1,"ID 0EF58FC2-90AC-77F0-85F2-91128B10372A Type Asset Path Example >> Cooling Tower 1 Asset Area A Name: 0, dtype: object"
1,0EF58FC2-90AC-77F0-8FD2-9F21B57B5B16,Example >> Cooling Tower 2,Area E,Area E,NaN,Asset,NaN,Example Data,False,1,"ID 0EF58FC2-90AA-60E0-94F7-29D4E89DC2F9 Type Asset Path Example >> Cooling Tower 1 Asset Area B Name: 1, dtype: object"
0,0EF58FC2-90AA-60E0-BEE7-242FC3F918D2,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False,2,"ID 0EF58FC2-90AC-77F0-85F2-91128B10372A Type Asset Path Example >> Cooling Tower 1 Asset Area A Name: 0, dtype: object"
2,0EF58FC2-90AC-77F0-9C25-FE38E1BE65AC,Example >> Cooling Tower 2,Area F,Area F,NaN,Asset,NaN,Example Data,False,2,"ID 0EF58FC2-90AA-60E0-94F7-29D4E89DC2F9 Type Asset Path Example >> Cooling Tower 1 Asset Area B Name: 1, dtype: object"


Now we swap! Note that we're going to have two new swapped calculations -- one for each Swap Group. You can see that there are a couple of actions noted in the `Swap Performed` column (separated by a `\n` line break).

In [11]:
swapped_df = spy.swap(added_together_df, swap_groups_df, old_asset_format=False)

# Use a specific set of display properties so that the linebreaks are rendered correctly
display(swapped_df[['ID', 'Name', 'Result', 'Swap Performed']].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}))

,Original ID,ID,Type,Name,Result,Swap Performed,Header
0,0EF5BF17-25A2-7110-8D07-FF485DDCEDB7,0EF5BF17-2D8C-66F0-9D31-517D1EBC8DED,CalculatedSignal,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area DExample >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area E,"Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area E"
1,0EF5BF17-25A2-7110-8D07-FF485DDCEDB7,0EF5BF17-2D8C-66F0-9FE8-AEFD5505D999,CalculatedSignal,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area DExample >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area F,"Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area F"


,ID,Name,Result,Swap Performed
0,0EF5BF17-2D8C-66F0-9D31-517D1EBC8DED,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D Example >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area E
1,0EF5BF17-2D8C-66F0-9FE8-AEFD5505D999,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D Example >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area F


We can pull just like we did before:

In [12]:
spy.pull(swapped_df, start='2021-01-01', end='2021-01-02')

,ID,Type,Name,Header,Time,Count,Pages,Data Processed,Result
0,0EF5BF17-2D8C-66F0-9D31-517D1EBC8DED,CalculatedSignal,Added Together,"Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area E",00:00:00.09,97,1,11 KB,Success
1,0EF5BF17-2D8C-66F0-9FE8-AEFD5505D999,CalculatedSignal,Added Together,"Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area F",00:00:00.08,97,1,11 KB,Success


,"Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area E","Cooling Tower 2 >> Area D, Cooling Tower 2 >> Area F"
2021-01-01 00:00:00+00:00,29.362794,28.016513
2021-01-01 00:15:00+00:00,32.655979,28.082996
2021-01-01 00:30:00+00:00,28.149479,28.149479
2021-01-01 00:45:00+00:00,32.784786,28.215962
2021-01-01 01:00:00+00:00,32.813098,28.282445
...,...,...
2021-01-01 23:00:00+00:00,27.369611,27.335406
2021-01-01 23:15:00+00:00,32.330629,27.660053
2021-01-01 23:30:00+00:00,28.261489,28.236783
2021-01-01 23:45:00+00:00,32.405502,27.776533


## Multi-level Swaps

If you have a hierarchy that contains "commonly named" nodes at the bottom of the tree, you'll need to do a multi-level swap. Here's an example where "Raw" and "Cleansed" are categorical child "assets" that are common between the uniquely-named assets (Area A & Area B):

```
Multi-level Swap Example >> Area A >> Raw      >> Temperature
                                      Cleansed >> Temperature
                            Area B >> Raw      >> Temperature
                                      Cleansed >> Temperature
```

You must specify the uniquely-named assets as your "Swap Out" values, and SPy will figure out how to perform the swap at the lower level. Here's an example where the categories are _Raw_ and _Cleansed_:

In [13]:
# First we have to create the asset tree we want from Example data.

area_ab_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': '/Area [AB]/',
    'Name': 'Temperature'
}, old_asset_format=False)

raw_signals = area_ab_df.copy()
raw_signals['Path'] = 'Multi-level Swap Example >> ' + raw_signals['Asset']
raw_signals['Asset'] = 'Raw'
raw_signals['Reference'] = True

cleansed_signals = area_ab_df.copy()
cleansed_signals['Path'] = 'Multi-level Swap Example >> ' + cleansed_signals['Asset']
cleansed_signals['Asset'] = 'Cleansed'
cleansed_signals['Reference'] = True

combined_signal = pd.DataFrame([{
    'Name': 'Raw and Cleansed Averaged',
    'Type': 'Signal',
    'Formula': '($r + $c) / 2',
    'Formula Parameters': {
        '$r': {'Path': 'Multi-level Swap Example >> Area A', 'Asset': 'Raw',      'Name': 'Temperature'},
        '$c': {'Path': 'Multi-level Swap Example >> Area A', 'Asset': 'Cleansed', 'Name': 'Temperature'}
    }
}])

pushed_items_df = spy.push(metadata=pd.concat([raw_signals, cleansed_signals, combined_signal], ignore_index=True),
                           workbook='SPy Documentation Examples >> spy.swap')

pushed_items_df[['ID', 'Type', 'Path', 'Asset', 'Name']]

,ID,Type,Path,Asset,Name
0,0EF5BF17-3AE3-FF60-A244-0B35E0F1FA9A,CalculatedSignal,Multi-level Swap Example >> Area A,Raw,Temperature
1,0EF5BF17-3BF0-E840-833B-3A01274979EE,CalculatedSignal,Multi-level Swap Example >> Area B,Raw,Temperature
2,0EF5BF17-3C34-EE00-993E-6E9C7100E2E9,CalculatedSignal,Multi-level Swap Example >> Area A,Cleansed,Temperature
3,0EF5BF17-4000-E8E0-996B-4C867B5969DB,CalculatedSignal,Multi-level Swap Example >> Area B,Cleansed,Temperature
4,0EF5BF17-40F2-6410-A74C-0261F5EE356F,CalculatedSignal,NaN,NaN,Raw and Cleansed Averaged
__side_effect_asset_1__,0EF5BF17-4049-ECC0-BF3B-42081211FDA8,Asset,,Multi-level Swap Example,Multi-level Swap Example
__side_effect_asset_2__,0EF5BF17-404E-EAE0-81E2-BED0CCC6507B,Asset,Multi-level Swap Example,Area A,Area A
__side_effect_asset_3__,0EF5BF17-404E-EAE0-813E-837416D86FBB,Asset,Multi-level Swap Example >> Area A,Raw,Raw
__side_effect_asset_4__,0EF5BF17-404E-EAE0-B533-FC0592C89204,Asset,Multi-level Swap Example,Area B,Area B
__side_effect_asset_5__,0EF5BF17-404E-EAE0-9383-4030C7678323,Asset,Multi-level Swap Example >> Area B,Raw,Raw


In [14]:
raw_plus_cleansed = pushed_items_df[pushed_items_df['Name'] == 'Raw and Cleansed Averaged']
area_a_df = pushed_items_df[(pushed_items_df['Type'] == 'Asset') & (pushed_items_df['Name'] == 'Area A')]
area_b_df = pushed_items_df[(pushed_items_df['Type'] == 'Asset') & (pushed_items_df['Name'] == 'Area B')]

# Create a Swap In / Swap Out pairing that uses the unique level of the tree, namely:
#  Swap Out:  test_multilevel_swap >> Example >> Cooling Tower 1 >> Area A
#  Swap In:   test_multilevel_swap >> Example >> Cooling Tower 1 >> Area B

assets_df = area_b_df.copy()                  # Swap In
assets_df['Swap Out'] = [area_a_df.iloc[0]]   # Swap Out

swap_results = spy.swap(raw_plus_cleansed, assets_df)

display(swap_results[['ID', 'Name', 'Swap Performed']].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}))

,Original ID,ID,Type,Name,Result,Swap Performed,Header
0,0EF5BF17-40F2-6410-A74C-0261F5EE356F,0EF5BF17-491C-7190-A57D-F9BC640E189B,CalculatedSignal,Raw and Cleansed Averaged,Success,Multi-level Swap Example >> Area A >> Raw --> Multi-level Swap Example >> Area B >> RawMulti-level Swap Example >> Area A >> Cleansed --> Multi-level Swap Example >> Area B >> Cleansed,"Area B >> Raw, Area B >> Cleansed"


,ID,Name,Swap Performed
0,0EF5BF17-491C-7190-A57D-F9BC640E189B,Raw and Cleansed Averaged,Multi-level Swap Example >> Area A >> Raw --> Multi-level Swap Example >> Area B >> Raw Multi-level Swap Example >> Area A >> Cleansed --> Multi-level Swap Example >> Area B >> Cleansed


## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [15]:
help(spy.swap)

Help on function swap in module seeq.spy._swap:

swap(items: pandas.core.frame.DataFrame, assets: pandas.core.frame.DataFrame, *, partial_swaps_ok: bool = False, old_asset_format: Optional[bool] = None, errors: Optional[str] = None, quiet: Optional[bool] = None, status: Optional[seeq.spy._status.Status] = None, session: Optional[seeq.spy._session.Session] = None)
    Operates on a DataFrame of items by swapping out the assets that those
    items are based on. The returned DataFrame can be supplied to
    spy.pull() to retrieve the resulting data.
    
    Parameters
    ----------
    items : {pd.DataFrame}
        A DataFrame of items over which to perform the swapping operation. The
        only required column is ID. Typically, you will have derived this
        DataFrame via a spy.search() or spy.push(metadata) call.
    
    assets : {pd.DataFrame}
        A DataFrame of Asset items (and ONLY Asset items) to swap IN. Each row
        must have valid ID, Type, Path, Asset, and Nam